In [1]:
import pandas as pd

### Dataset Overview
This notebook merges data from the following sources:
* UNHCR Population Statistics - Asylum Seekers (http://popstats.unhcr.org/en/asylum_seekers):  UNHCR served as the source for the dependent variable: Number of new asylum seekers per country per year.
* Worldbank (https://data.worldbank.org/indicator/NY.GDP.MKTP.CD): Historical GDP data, population by country, life expectancy, unemployment
* GDELT Project (https://www.gdeltproject.org/data.html): Database of global events, specifically around political unrest and conflict.
* ISO codes, FIPS codes: Used to join across datasets 

In [2]:
# 1. Load data
# (a) Read in asylum seekers
seekers = pd.read_csv('Target_Variable/Asylum_seekers_per_country_per_year_w_prev_year.csv')
seekers = seekers.drop('Unnamed: 0', axis = 1)

print(seekers.shape)
seekers.head()

(3843, 10)


,Year,Origin,Applied during year,year_origin,English_short_name,French_short_name,Alpha_2_code,Alpha_3_code,Numeric,applied_previous_year
0,2000,Afghanistan,291283,"(2000, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,0.0
1,2000,Albania,9765,"(2000, 'Albania')",Albania,Albanie (l'),AL,ALB,8.0,0.0
2,2000,Algeria,10312,"(2000, 'Algeria')",Algeria,Algérie (l'),DZ,DZA,12.0,0.0
3,2000,Andorra,1,"(2000, 'Andorra')",Andorra,Andorre (l'),AD,AND,20.0,0.0
4,2000,Angola,7160,"(2000, 'Angola')",Angola,Angola (l'),AO,AGO,24.0,0.0


In [3]:
# (b) Read in ISO codes
iso = pd.read_csv('Target_Variable/ISO_codes.csv')

iso.head()

,English_short_name,French_short_name,Alpha_2_code,Alpha_3_code,Numeric,Origin
0,Afghanistan,Afghanistan (l'),AF,AFG,4,Afghanistan
1,Albania,Albanie (l'),AL,ALB,8,Albania
2,Algeria,Algérie (l'),DZ,DZA,12,Algeria
3,American Samoa,Samoa américaines (les),AS,ASM,16,American Samoa
4,Andorra,Andorre (l'),AD,AND,20,Andorra


In [4]:
# (c) Read in populations
pop = pd.read_csv('initial_analysis/populations.csv', skiprows = [i for i in range(0, 3)])

pop.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,NaN


In [5]:
# (d) Read in economic stats
econ = pd.read_excel('Economic data clean.xlsx', sheet_name = 'Asylum_seekers_per_country_per_')
econ = econ.drop('Unnamed: 0', axis = 1)

econ.head()

,Year,Origin,Clean_name,Applied during year,year_origin,applied_previous_year,Unemployment Rate,GDP Per Capita,Life expectancy,GDP Per Capita (Clean),Unemployment (Clean),Life expactancy (clean)
0,2000,Afghanistan,Afghanistan,291283,"(2000, 'Afghanistan')",0.0,3.517,0.000000,55.841,179.426494,3.517,55.841
1,2000,Albania,Albania,9765,"(2000, 'Albania')",0.0,17.767,1126.683318,73.955,1126.683318,17.767,73.955
2,2000,Algeria,Algeria,10312,"(2000, 'Algeria')",0.0,29.770,1765.022577,70.640,1765.022577,29.770,70.640
3,2000,Andorra,Andorra,1,"(2000, 'Andorra')",0.0,0.000,21936.530101,0.000,21936.530101,5.477,67.549
4,2000,Angola,Angola,7160,"(2000, 'Angola')",0.0,22.885,556.836318,46.522,556.836318,22.885,46.522


In [6]:
# (e) Read in FIPS 
fips = pd.read_csv('initial_analysis/fips.csv')

fips.head()

,FIPS 10-4,ISO 3166,Name
0,AF,AF,Afghanistan
1,AX,-,Akrotiri
2,AL,AL,Albania
3,AG,DZ,Algeria
4,AQ,AS,American Samoa


In [7]:
# (f) Read in gdelt v2
gdelt = pd.read_csv('gdelt/gdelt_dataframe_v3.csv')

gdelt.head()

,Year,Actor2Geo_CountryCode,GoldsteinScale,NuMentions,AvgTone,QuadClass,Goldstein_Neg_5,Goldstein_Neg_6,Goldstein_Neg_7,Goldstein_Neg_8,...,Goldstein_Pos_8,Goldstein_Pos_9,Quad_4_Goldstein_Neg_5,Quad_4_Goldstein_Neg_6,Quad_4_Goldstein_Neg_7,Quad_4_Goldstein_Neg_8,Quad_4_Goldstein_Neg_9,Event_Code_19_20,Extreme_Pos_Tone_Events,Extreme_Neg_Tone_Events
0,2000,AA,0.325581,4.139535,4.938495,1.790698,4,3,2,1,...,0,0,2,1,1,1,1,1,0,0
1,2000,AC,-0.314729,4.147287,4.402594,1.992248,16,15,15,15,...,0,0,15,15,15,15,14,12,4,0
2,2000,AE,2.149691,4.211136,5.776108,1.391738,171,171,151,142,...,20,3,157,157,151,142,130,116,305,0
3,2000,AF,0.438194,4.964461,4.805762,1.862716,3989,3942,3711,3296,...,273,132,3726,3719,3691,3296,3133,3003,371,0
4,2000,AG,0.546953,5.070051,5.395546,1.786704,2722,2715,2481,2404,...,139,35,2580,2578,2476,2404,2210,2000,847,0


In [8]:
# 2. Merge
# (a) Merge seekers with iso.
seekers_iso = seekers.merge(iso, left_on = 'Origin', right_on = 'Origin', how = 'outer', indicator = True)
seekers_iso.rename(columns = {'_merge': 'iso_merge'}, inplace = True)

In [9]:
seekers_iso.head()

,Year,Origin,Applied during year,year_origin,English_short_name_x,French_short_name_x,Alpha_2_code_x,Alpha_3_code_x,Numeric_x,applied_previous_year,English_short_name_y,French_short_name_y,Alpha_2_code_y,Alpha_3_code_y,Numeric_y,iso_merge
0,2000.0,Afghanistan,291283.0,"(2000, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,0.0,Afghanistan,Afghanistan (l'),AF,AFG,4.0,both
1,2001.0,Afghanistan,66697.0,"(2001, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,291283.0,Afghanistan,Afghanistan (l'),AF,AFG,4.0,both
2,2002.0,Afghanistan,31781.0,"(2002, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,66697.0,Afghanistan,Afghanistan (l'),AF,AFG,4.0,both
3,2003.0,Afghanistan,22856.0,"(2003, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,31781.0,Afghanistan,Afghanistan (l'),AF,AFG,4.0,both
4,2004.0,Afghanistan,14893.0,"(2004, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,22856.0,Afghanistan,Afghanistan (l'),AF,AFG,4.0,both


In [10]:
# (b) Merge seekers_iso with pop.
seekers_iso_pop = seekers_iso.merge(pop, left_on = 'Alpha_3_code_x', right_on = 'Country Code', how = 'outer', indicator = True)
seekers_iso_pop.rename(columns = {'_merge': 'pop_merge'}, inplace = True)

In [11]:
seekers_iso_pop.head()

,Year,Origin,Applied during year,year_origin,English_short_name_x,French_short_name_x,Alpha_2_code_x,Alpha_3_code_x,Numeric_x,applied_previous_year,...,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63,pop_merge
0,2000.0,Afghanistan,291283.0,"(2000, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,0.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN,both
1,2001.0,Afghanistan,66697.0,"(2001, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,291283.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN,both
2,2002.0,Afghanistan,31781.0,"(2002, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,66697.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN,both
3,2003.0,Afghanistan,22856.0,"(2003, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,31781.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN,both
4,2004.0,Afghanistan,14893.0,"(2004, 'Afghanistan')",Afghanistan,Afghanistan (l'),AF,AFG,4.0,22856.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN,both


In [12]:
# (c) Merge seekers_iso_pop with econ.
seekers_iso_pop_econ = seekers_iso_pop.merge(econ, left_on = ['English_short_name_x', 'Year'], right_on = ['Clean_name', 'Year'], how = 'outer', indicator = True)
seekers_iso_pop_econ.rename(columns = {'_merge': 'econ_merge'}, inplace = True)

In [13]:
# (d) Merge seekers_iso with fips
seekers_iso_pop_econ_fips = seekers_iso_pop_econ.merge(fips, left_on = 'Alpha_2_code_x', right_on = 'ISO 3166', how = 'outer', indicator = True)
seekers_iso_pop_econ_fips.rename(columns = {'_merge': 'fips_merge'}, inplace = True)

In [14]:
# (e) Merge seekers_iso_pop with gdelt.
seekers_iso_pop_econ_fips_gdelt = seekers_iso_pop_econ_fips.merge(gdelt, left_on = ['FIPS 10-4', 'Year'], right_on = ['Actor2Geo_CountryCode', 'Year'], how = 'outer', indicator = True)
seekers_iso_pop_econ_fips_gdelt.rename(columns = {'_merge': 'gdelt_merge'}, inplace = True)

In [15]:
# (f) Check merge.
print(seekers_iso_pop_econ_fips_gdelt.groupby(['fips_merge', 'iso_merge', 'gdelt_merge', 'econ_merge', 'pop_merge']).size())

fips_merge  iso_merge   gdelt_merge  econ_merge  pop_merge
both        left_only   both         left_only   left_only      57
            right_only  left_only    left_only   left_only      30
            both        left_only    both        both           62
                        both         left_only   left_only      19
                                     both        left_only      77
                                                 both         3768
dtype: int64


In [16]:
seekers_iso_pop_econ_fips_gdelt[
    (seekers_iso_pop_econ_fips_gdelt['fips_merge'] == 'both') & 
    (seekers_iso_pop_econ_fips_gdelt['pop_merge']  == 'left_only') & 
    (seekers_iso_pop_econ_fips_gdelt['iso_merge']  == 'left_only')].Year.value_counts().sort_index()

2000.0    2
2001.0    3
2002.0    3
2003.0    2
2004.0    3
2005.0    3
2006.0    3
2007.0    3
2008.0    3
2009.0    3
2010.0    2
2011.0    3
2012.0    3
2013.0    4
2014.0    3
2015.0    3
2016.0    4
2017.0    4
2018.0    3
Name: Year, dtype: int64

In [17]:
sum(seekers_iso_pop_econ_fips_gdelt.duplicated())

0

In [18]:
# There were 31 rows with Year = NaN, this removes them.
seekers_iso_pop_econ_fips_gdelt = seekers_iso_pop_econ_fips_gdelt.dropna(subset=['Year'])

In [19]:
# 3. Export
seekers_iso_pop_econ_fips_gdelt.to_csv('merged_seekers_iso_pop_gdelt_v4.csv')